In [16]:
import os
filenames = []
for root, dirs, files in os.walk("saved_plays", topdown=True):
    for name in files:
        filenames.append(os.path.join(root, name))

In [20]:
import pandas as pd

filenames = [f for f in filenames if 'DS_Store' not in f]
df = pd.DataFrame({'filename': filenames})

In [21]:
df['side'] = df['filename'].str.split('/').apply(lambda x: x[1])

In [22]:
df.head()

,filename,side
0,saved_plays/right/pass/Madden NFL 19_201903151...,right
1,saved_plays/right/pass/Madden NFL 19_201904031...,right
2,saved_plays/right/pass/Madden NFL 19_201905011...,right
3,saved_plays/right/pass/Madden NFL 19_201903232...,right
4,saved_plays/right/pass/Madden NFL 19_201904271...,right


In [23]:
#df.to_csv('side_dataset.csv')

In [66]:
from torch.utils.data import Dataset
import cv2 as cv

# Make Dataset class
label_map = {'left': 0, 'neither': 1, 'right': 2}

class PlaySideDataset(Dataset):
    
    def __init__(self, csv_file, transform=None):
        '''
        '''
        self.dataframe = pd.read_csv(csv_file, index_col=0)
        self.transform = transform
        self.size = (300,100)
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        '''
        '''
        img_name = self.dataframe.iloc[idx, 0]
        
        image = cv.imread(img_name)
        #image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        #
        image = cv.resize(image, self.size)

        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        
        return (image, label_map[label])

In [67]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import numpy as np

validation_indices = []

def get_training_and_validation_loaders(dataset, batch_size = 16, validation_split = .2):
    
    shuffle_dataset = True
    random_seed = 8

    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))

    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
        
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)
    #
    validation_indices = val_indices
    
    return train_loader, validation_loader

In [68]:
from torchvision import transforms

trans = transforms.Compose([transforms.ToTensor(), 
                            transforms.Normalize((0.5,), (0.5,)),
                           ])

dset = PlaySideDataset('side_dataset.csv', trans)

In [69]:
x, y = dset[0]
x.shape

torch.Size([3, 100, 300])

In [70]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 
train_loader, validation_loader = get_training_and_validation_loaders(dset, batch_size=15)

In [71]:
batch, labels = next(iter(train_loader))

In [75]:
import torch.nn as nn
import torch.nn.functional as F

conv1 = nn.Conv2d(3, 16, kernel_size=5)
x = F.relu(F.max_pool2d(conv1(batch), 2))
print(x.shape)
# torch.Size([15, 16, 48, 148])

conv2 = nn.Conv2d(16, 32, kernel_size=5)
x = F.relu(F.max_pool2d(conv2(x), 2))
print(x.shape)
# torch.Size([15, 32, 22, 72])

conv3 = nn.Conv2d(32, 64, kernel_size=5)
x = F.relu(F.max_pool2d(conv3(x), 2))
print(x.shape)
# torch.Size([15, 64, 9, 34])

x = x.view(15, -1)
# torch.Size([15, 19584])

fc1 = nn.Linear(19584, 100)
x = F.relu(fc1(x))
torch.Size([15, 100])


torch.Size([15, 16, 48, 148])
torch.Size([15, 32, 22, 72])
torch.Size([15, 64, 9, 34])
torch.Size([15, 100])


In [74]:
x.view(15, -1).shape

torch.Size([15, 19584])

In [80]:
# BNN
#

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(19584, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #print(x.shape) # [N, 64, 72, 42]
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        #print(x.shape) # [N, 128, 34, 19]
        #x = F.relu(F.max_pool2d(x, 2))
        #print(x.shape) # [N, 128, 17, 9]
        x = x.view(-1, 19584)
        #print(x.shape) # 
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #print(x.shape)
        return F.log_softmax(x, dim=1)

    def name(self):
        return "mutclipside_may19"

In [81]:
# HPs
lr = 0.01 # 0.001 too small

# Model
model = Net().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum=0.9)

loss_list = []
acc_list = []
train_losses = []
train_accuracies = []
train_losses2 = []
#
num_epochs = 40

for epoch in range(num_epochs):
        
    # Train:   
    total = 0
    correct = 0
    trainlosses = []
    
    for batch_index, (images, labels) in enumerate(train_loader):
        # Send to GPU (device)
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images.float())
        
        # Loss
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        trainlosses.append(loss.item())
        
        # Compute accuracy
        _, argmax = torch.max(outputs, 1)
        correct += (labels == argmax).sum().item() #.mean()
        total += len(labels)
        
    train_losses.append( np.mean(trainlosses) )
    train_accuracies.append( correct/total )
    train_losses2.extend(trainlosses)
                        
    with torch.no_grad():
        
        losses = []
        total = 0
        correct = 0
    
        for images, labels in validation_loader:
            # 
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images.float())
            
            loss = criterion(outputs, labels)
        
            # Compute accuracy
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item() #.mean()
            total += len(labels)
            
            losses.append(loss.item())
    
        acc = np.round(correct/total, 3)
        loss = np.mean(losses)
        
        acc_list.append(acc)
        loss_list.append(loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {np.mean(losses):.4}, Acc: {correct/total:.2}')

Epoch [1/40], Loss: 0.7503, Acc: 0.73
Epoch [2/40], Loss: 0.2464, Acc: 0.86
Epoch [3/40], Loss: 0.1873, Acc: 0.95


KeyboardInterrupt: 